In [1]:
# !pip install pyarrow

In [2]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

In [3]:
df = pd.read_parquet("../data/dataset.parquet")
df

,date,open,high,low,close,adj close,volume,close_percent_change,dividends_event_value,dividends_event_name,...,volume_lag_55,volume_lag_56,volume_lag_57,volume_lag_58,volume_lag_59,day_of_week,day_of_month,month,year,week_of_year
0,2014-08-06,23.687500,23.870001,23.677500,23.740000,20.998869,154232000,-0.005055,0.0000,None,...,NaN,NaN,NaN,NaN,NaN,2,6,8,2014,32
1,2014-08-07,23.732500,23.987499,23.525000,23.620001,20.996649,186844000,0.002752,0.1175,dividends,...,NaN,NaN,NaN,NaN,NaN,3,7,8,2014,32
2,2014-08-08,23.565001,23.705000,23.320000,23.684999,21.054420,167460000,0.013194,0.1175,dividends,...,NaN,NaN,NaN,NaN,NaN,4,8,8,2014,32
3,2014-08-11,23.817499,24.020000,23.709999,23.997499,21.332216,146340000,-0.000208,0.1175,dividends,...,NaN,NaN,NaN,NaN,NaN,0,11,8,2014,33
4,2014-08-12,24.010000,24.219999,23.902500,23.992500,21.327774,135180000,0.013233,0.1175,dividends,...,NaN,NaN,NaN,NaN,NaN,1,12,8,2014,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,2024-07-30,219.190002,220.330002,216.119995,218.800003,218.800003,41643800,0.014991,0.2500,dividends,...,48983000.0,45057100.0,77305800.0,78569700.0,163224100.0,1,30,7,2024,31
2512,2024-07-31,221.440002,223.820007,220.630005,222.080002,222.080002,50036300,-0.016751,0.2500,dividends,...,50759500.0,48983000.0,45057100.0,77305800.0,78569700.0,2,31,7,2024,31
2513,2024-08-01,224.369995,224.479996,217.020004,218.360001,218.360001,62501000,0.006869,0.2500,dividends,...,72044800.0,50759500.0,48983000.0,45057100.0,77305800.0,3,1,8,2024,31
2514,2024-08-02,219.149994,225.600006,217.710007,219.860001,219.860001,105568600,-0.048167,0.2500,dividends,...,52393600.0,72044800.0,50759500.0,48983000.0,45057100.0,4,2,8,2024,31


In [4]:
# target = "target_close"
target = "close_percent_change"

x = df.drop(columns=[target]).set_index("date")
y = df[["date", target]].set_index("date")
y

,close_percent_change
date,
2014-08-06,-0.005055
2014-08-07,0.002752
2014-08-08,0.013194
2014-08-11,-0.000208
2014-08-12,0.013233
...,...
2024-07-30,0.014991
2024-07-31,-0.016751
2024-08-01,0.006869


In [5]:
x

,open,high,low,close,adj close,volume,dividends_event_value,dividends_event_name,dividends_event_sentiment,dividends_event_decay,...,volume_lag_55,volume_lag_56,volume_lag_57,volume_lag_58,volume_lag_59,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2014-08-06,23.687500,23.870001,23.677500,23.740000,20.998869,154232000,0.0000,None,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2,6,8,2014,32
2014-08-07,23.732500,23.987499,23.525000,23.620001,20.996649,186844000,0.1175,dividends,1.0,1.000000,...,NaN,NaN,NaN,NaN,NaN,3,7,8,2014,32
2014-08-08,23.565001,23.705000,23.320000,23.684999,21.054420,167460000,0.1175,dividends,1.0,0.980000,...,NaN,NaN,NaN,NaN,NaN,4,8,8,2014,32
2014-08-11,23.817499,24.020000,23.709999,23.997499,21.332216,146340000,0.1175,dividends,1.0,0.960400,...,NaN,NaN,NaN,NaN,NaN,0,11,8,2014,33
2014-08-12,24.010000,24.219999,23.902500,23.992500,21.327774,135180000,0.1175,dividends,1.0,0.941192,...,NaN,NaN,NaN,NaN,NaN,1,12,8,2014,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-30,219.190002,220.330002,216.119995,218.800003,218.800003,41643800,0.2500,dividends,1.0,0.335899,...,48983000.0,45057100.0,77305800.0,78569700.0,163224100.0,1,30,7,2024,31
2024-07-31,221.440002,223.820007,220.630005,222.080002,222.080002,50036300,0.2500,dividends,1.0,0.329181,...,50759500.0,48983000.0,45057100.0,77305800.0,78569700.0,2,31,7,2024,31
2024-08-01,224.369995,224.479996,217.020004,218.360001,218.360001,62501000,0.2500,dividends,1.0,0.322597,...,72044800.0,50759500.0,48983000.0,45057100.0,77305800.0,3,1,8,2024,31


In [6]:
table_info = pd.DataFrame()

# Print columns that are not int, float, bool or category
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        table_info = pd.concat([table_info, pd.DataFrame({"Column Name": [col], "Data Type": [x[col].dtype]})])

table_info

,Column Name,Data Type
0,volume,int64
0,dividends_event_name,object
0,splits_event_name,object


In [7]:
# Convert columns to float if they are not int, float, bool or category. Handle Cannot cast DatetimeArray to dtype float64
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        try:
            x[col] = x[col].astype("float")
        except:
            # drop datetime columns
            x = x.drop(columns=[col])

            print(f"Dropped Column: {col}")

Dropped Column: dividends_event_name
Dropped Column: splits_event_name


In [8]:
# # Normalizing the features between 0 and 1
# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# x_scaler = MinMaxScaler()
# x = x_scaler.fit_transform(x)

# y

In [9]:
#    All Features   | Target
# +-----------------+---------+
# | x_train         | y_train | <- 85% of the data which is used for training
# |                 |         |
# +-----------------+---------+
# | x_test          | y_test  | <- 15% of the data which is used for testing
# +-----------------+---------+



# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True)
x_train

,open,high,low,close,adj close,volume,dividends_event_value,dividends_event_sentiment,dividends_event_decay,splits_event_value,...,volume_lag_55,volume_lag_56,volume_lag_57,volume_lag_58,volume_lag_59,day_of_week,day_of_month,month,year,week_of_year
date,,,,,,,,,,,,,,,,,,,,,
2021-03-10,121.690002,122.169998,119.449997,119.980003,117.763794,111943300.0,0.2050,1.0,0.641171,4.0,...,94359800.0,98208600.0,157243700.0,79184500.0,86939800.0,2,10,3,2021,10
2022-03-04,164.490005,165.550003,162.100006,163.169998,161.107880,83737200.0,0.2200,1.0,0.681233,4.0,...,139380400.0,153237000.0,115402700.0,108923700.0,116998900.0,4,4,3,2022,9
2015-01-30,29.600000,30.000000,29.212500,29.290001,26.149811,334982000.0,0.1175,1.0,0.316145,0.0,...,108782000.0,134766000.0,139874000.0,149743600.0,166297600.0,4,30,1,2015,5
2022-05-04,159.669998,166.479996,159.259995,166.020004,163.921860,108256500.0,0.2200,1.0,0.291602,4.0,...,86185500.0,98670700.0,90865900.0,71285000.0,74829200.0,2,4,5,2022,18
2023-07-25,193.330002,194.440002,192.919998,193.619995,192.597153,37283200.0,0.2400,1.0,0.371602,4.0,...,81235400.0,65136000.0,48425700.0,52472900.0,55209200.0,1,25,7,2023,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-21,26.820000,26.842501,26.392500,26.832500,24.363945,190362400.0,0.1300,1.0,0.534575,0.0,...,232079200.0,255716400.0,265892000.0,293461600.0,208436000.0,0,21,12,2015,52
2021-11-26,159.570007,160.449997,156.360001,156.809998,154.631256,76959800.0,0.2200,1.0,0.753642,4.0,...,57305700.0,74420200.0,82278300.0,57808700.0,71115500.0,4,26,11,2021,47
2020-07-24,90.987503,92.970001,89.144997,92.614998,90.449203,185438800.0,0.2050,1.0,0.342754,0.0,...,142333600.0,147751200.0,133568000.0,240616800.0,183064000.0,4,24,7,2020,30


In [10]:
y_test

,close_percent_change
date,
2022-08-08,0.000303
2017-04-21,0.009630
2022-10-07,0.002356
2016-03-11,0.002542
2019-12-03,0.008826
...,...
2018-10-24,0.021898
2018-07-19,-0.002293
2016-05-26,-0.000598


In [11]:
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1, colsample_bytree=0.8)
model = xgb.XGBRegressor(n_estimators=150, max_depth=7, eta=0.1, subsample=1, colsample_bytree=.5)
model.fit(x_train, y_train)

# pickle.dump(model, open("../model/xgboost_model.pkl", "wb")) # Save model as Python pickle object
model.save_model("../model/xgboost_model.json")

In [12]:
predicted = model.predict(x_test)
# predicted

In [13]:
# Add predicted values to the dataframe
y_test["predicted"] = predicted
y_test

,close_percent_change,predicted
date,,
2022-08-08,0.000303,0.004119
2017-04-21,0.009630,0.005950
2022-10-07,0.002356,0.005257
2016-03-11,0.002542,0.003434
2019-12-03,0.008826,0.010803
...,...,...
2018-10-24,0.021898,-0.002282
2018-07-19,-0.002293,0.007985
2016-05-26,-0.000598,0.002250


In [14]:
# Get the root mean squared error (RMSE)
mse = np.mean((y_test["predicted"] - y_test[target]) ** 2)
rmse = np.sqrt(mse)
rmse

0.01814530471832343

In [15]:
# Get the mean absolute error (MAE)
mae = np.mean(np.abs(y_test["predicted"] - y_test[target]))
mae

0.013045830621112642

In [16]:
# Get the mean absolute percentage error (MAPE)
mape = np.mean(np.abs((y_test[target] - y_test["predicted"]) / y_test[target])) * 100
mape

373.49991614137286

In [18]:
# Save predicted values
y_test.sort_index().to_parquet("../data/predicted.parquet")